In [8]:
import tensorflow as tf
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import utils
import scoring
import numpy as np

from skopt import Optimizer

from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, RBF, WhiteKernel

from skopt.learning import RandomForestRegressor

from skopt.acquisition import gaussian_ei as acq_func

from IPython import display

Using TensorFlow backend.


In [2]:
### Importing and structuring data ###

DATA_PATH = "./data/"
train, test = utils.load_small_data_csv(DATA_PATH,"train_smaller100.csv.gz" , "test_smaller100.csv.gz", utils.SIMPLE_FEATURE_COLUMNS)

PointResiduals,Angles = utils.kink(train)
train['PointResiduals'] = pd.Series(PointResiduals, index=train.index)
train['Angles'] = pd.Series(PointResiduals, index=train.index)


train_part, val_part = train_test_split(train, test_size=0.20, shuffle=True)
x_train = train_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values
x_val   =  val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values
y_train = train_part.loc[:, ["label"]].values
y_val = val_part.loc[:, ["label"]].values
#y_train_weight = train_part.loc[:, ["weight"]].values

# turn labels into categorical classes
classes = [0,1]
y_train = keras.utils.to_categorical(y_train, num_classes=len(classes))
y_val   = keras.utils.to_categorical(y_val,   num_classes=len(classes))

In [3]:
### Defining utils ###

# Rectified linear unit

def relu(x):
  return np.array([ (i>0) * abs(i) for i in x ])

# plotting the bayesian optimizer

def plot_bo(bo, suggestion=None, value=None):
    a, b = bo.space.bounds[0]
    
    # getting the latest model
    model = bo.models[-1]
    
    xs = np.linspace(a, b, num=100)
    x_model = bo.space.transform(xs.reshape(-1, 1).tolist())
    
    mean, std = model.predict(x_model, return_std=True)
    
    plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.scatter(
        np.array(bo.Xi)[:, 0],
        np.array(bo.yi),
        color='red',
        label='observations'
    )
    if suggestion is not None:
        plt.scatter([suggestion], value, color='blue', label='suggestion')
    
    plt.plot(xs, mean, color='green', label='model')
    plt.fill_between(xs, mean - 1.96 * std, mean + 1.96 * std, alpha=0.1, color='green')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    acq = acq_func(x_model, model, np.min(bo.yi))
    plt.plot(xs, acq, label='Expected Improvement')
    plt.legend()
    
    plt.show()

# 

def cum_min(xs):
    result = np.zeros_like(xs)
    cmin = xs[0]
    
    result[0] = xs[0]
    
    for i in range(1, xs.shape[0]):
        if cmin > xs[i]:
            cmin = xs[i]

        result[i] = cmin
    
    return result

# plots progress of BO over time

def plot_convergence(bo):
    display.clear_output(wait=True)
    values = np.array(bo.yi)
    
    plt.figure(figsize=(12, 6))
    plt.plot(cum_min(values), label='minimal discovered')
    plt.scatter(np.arange(len(bo.yi)), bo.yi, label='observations')
    plt.xlabel('step', fontsize=14)
    plt.ylabel('loss', fontsize=14)
    
    plt.legend(loc='upper right', fontsize=18)
    
    plt.show()

# Prints best parameters
    
def print_best(bo):
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]

    NodesInFirstDense, NodesInSecondDense, DropoutValue, INIT_LEARNINGRATE = best_parameters
    
    print(
        'Best model:\n Nodes in first dense layer= {0} \n Nodes in second dense layer= {1} \n learning rate= {2} \n Dropout value= {3}'.format(
            int(np.ceil(NodesInFirstDense)),
            int(np.ceil(NodesInSecondDense)),
            INIT_LEARNINGRATE,
            DropoutValue
        )
    )

In [4]:
### Target function with as input optimizeable parameters ###

def target_function1(params, X_train=x_train, y_train=y_train, X_score=x_val, y_score=y_val):
    
    # Optimized parameters
    NodesInFirstDense, NodesInSecondDense, INIT_LEARNINGRATE, DropoutValue = params
    
    
    # Making sure that the number of nodes are integers
    NodesInFirstDense = int(np.ceil(NodesInFirstDense))
    NodesInSecondDense = int(np.ceil(NodesInSecondDense))
    
    # Two parameters not optimized in this case, but can be optimized if needed
    BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
    EPOCHS = 3

    assert len(y_train) == len(x_train), "x_train and y_train not same length!"
    #assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout
    from keras.layers.normalization import BatchNormalization

    K.clear_session()

    model = Sequential()
    model.add(Dense(NodesInFirstDense, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
    model.add(BatchNormalization())
    model.add(Dropout(DropoutValue))
    model.add(Dense(NodesInSecondDense , activation='relu'))
    model.add(Dense( len(classes) )) # muon and 'other'
    model.add(Activation("softmax")) # output probabilities

    model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
        metrics=['accuracy'] 
        )


    model.fit(
        x_train, y_train,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        validation_data = (x_val, y_val),
        shuffle = True
        )

    #model.save_model("keras_basic_model.xgb")


    # score

    validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
    result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
    print(result)
    
    return 1 - result

In [5]:
def target_function2(params, X_train=x_train, y_train=y_train, X_score=x_val, y_score=y_val):
    
    # Optimized parameters
    NodesInFirstDense, NodesInSecondDense, NodesInThirdDense, INIT_LEARNINGRATE, DropoutValue  = params
    
    # Making sure that the number of nodes are integers
    NodesInFirstDense = int(np.ceil(NodesInFirstDense))
    NodesInSecondDense = int(np.ceil(NodesInSecondDense))
    NodesInThirdDense = int(np.ceil(NodesInThirdDense))
    
    # Two parameters not optimized in this case, but can be optimized if needed
    BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
    EPOCHS = 3

    assert len(y_train) == len(x_train), "x_train and y_train not same length!"
    #assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout
    from keras.layers.normalization import BatchNormalization

    K.clear_session()

    model = Sequential()
    model.add(Dense(NodesInFirstDense, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
    model.add(BatchNormalization())
    model.add(Dropout(DropoutValue))
    model.add(Dense(NodesInSecondDense , activation='relu'))
    model.add(Dense(NodesInThirdDense )) # muon and 'other'
    model.add(Dense( len(classes) )) # muon and 'other'
    model.add(Activation("softmax")) # output probabilities

    model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
        metrics=['accuracy'] 
        )


    model.fit(
        x_train, y_train,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        validation_data = (x_val, y_val),
        shuffle = True
        )

    #model.save_model("keras_basic_model.xgb")


    # score

    validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
    result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
    print(result)
    
    return 1 - result

In [6]:
def target_function3(params, X_train=x_train, y_train=y_train, X_score=x_val, y_score=y_val):
    
    # Optimized parameters
    NodesInFirstDense, NodesInSecondDense, NodesInThirdDense, NodesInFourthDense, INIT_LEARNINGRATE, DropoutValue = params
    
    # Making sure that the number of nodes are integers
    NodesInFirstDense = int(np.ceil(NodesInFirstDense))
    NodesInSecondDense = int(np.ceil(NodesInSecondDense))
    NodesInThirdDense = int(np.ceil(NodesInThirdDense))
    NodesInFourthDense = int(np.ceil(NodesInFourthDense))
    
    # Two parameters not optimized in this case, but can be optimized if needed
    BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
    EPOCHS = 3

    assert len(y_train) == len(x_train), "x_train and y_train not same length!"
    #assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout
    from keras.layers.normalization import BatchNormalization

    K.clear_session()

    model = Sequential()
    model.add(Dense(NodesInFirstDense, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
    model.add(BatchNormalization())
    model.add(Dense(NodesInSecondDense , activation='relu'))
    model.add(Dense(NodesInThirdDense )) # muon and 'other'
    model.add(Dropout(DropoutValue))
    model.add(Dense(NodesInFourthDense )) # muon and 'other'
    model.add(Dense( len(classes) ))
    model.add(Activation("softmax")) # output probabilities

    model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
        metrics=['accuracy'] 
        )


    model.fit(
        x_train, y_train,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        validation_data = (x_val, y_val),
        shuffle = True
        )

    #model.save_model("keras_basic_model.xgb")


    # score

    validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
    result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
    print(result)
    
    return 1 - result

In [7]:
def target_function4(params, X_train=x_train, y_train=y_train, X_score=x_val, y_score=y_val):
    
    # Optimized parameters
    NodesInFirstDense, NodesInSecondDense, NodesInThirdDense, NodesInFourthDense,NodesInFifthDense, INIT_LEARNINGRATE, DropoutValue = params
    
    # Making sure that the number of nodes are integers
    NodesInFirstDense = int(np.ceil(NodesInFirstDense))
    NodesInSecondDense = int(np.ceil(NodesInSecondDense))
    NodesInThirdDense = int(np.ceil(NodesInThirdDense))
    NodesInFourthDense = int(np.ceil(NodesInFourthDense))
    NodesInFifthDense = int(np.ceil(NodesInFifthDense))
    
    # Two parameters not optimized in this case, but can be optimized if needed
    BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
    EPOCHS = 3

    assert len(y_train) == len(x_train), "x_train and y_train not same length!"
    #assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout
    from keras.layers.normalization import BatchNormalization

    K.clear_session()

    model = Sequential()
    model.add(Dense(NodesInFirstDense, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
    model.add(BatchNormalization())
    model.add(Dense(NodesInSecondDense , activation='relu'))
    model.add(Dense(NodesInThirdDense )) # muon and 'other'
    model.add(Dropout(DropoutValue))
    model.add(Dense(NodesInFourthDense )) # muon and 'other'
    model.add(Dense(NodesInFifthDense ))
    model.add(Dense( len(classes) ))
    model.add(Activation("softmax")) # output probabilities


    model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
        metrics=['accuracy'] 
        )


    model.fit(
        x_train, y_train,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        validation_data = (x_val, y_val),
        shuffle = True
        )

    #model.save_model("keras_basic_model.xgb")


    # score

    validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
    result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
    print(result)
    
    return 1 - result

In [8]:
def target_function_NoBatch(params, X_train=x_train, y_train=y_train, X_score=x_val, y_score=y_val):
    
    # Optimized parameters
    NodesInFirstDense, NodesInSecondDense, INIT_LEARNINGRATE, DropoutValue = params
    
    # Making sure that the number of nodes are integers
    NodesInFirstDense = int(np.ceil(NodesInFirstDense))
    NodesInSecondDense = int(np.ceil(NodesInSecondDense))
    
    # Two parameters not optimized in this case, but can be optimized if needed
    BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
    EPOCHS = 3

    assert len(y_train) == len(x_train), "x_train and y_train not same length!"
    #assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

    from keras.models import Sequential
    from keras.layers import Dense, Activation, Dropout

    K.clear_session()

    model = Sequential()
    model.add(Dense(NodesInFirstDense, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
    model.add(Dropout(DropoutValue))
    model.add(Dense(NodesInSecondDense , activation='relu'))
    model.add(Dense( len(classes) ))
    model.add(Activation("softmax")) # output probabilities

    model.compile(
        loss="categorical_crossentropy",
        optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
        metrics=['accuracy'] 
        )


    model.fit(
        x_train, y_train,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        validation_data = (x_val, y_val),
        shuffle = True
        )

    #model.save_model("keras_basic_model.xgb")


    # score

    validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
    result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
    print(result)
    
    return 1 - result

In [9]:
### Setting dimensions for optimizeable parameters ###

dimensions_4 =[
    # NodesInFirstDense
    (4.0, 300.0),
    
    # NodesInSecondDense
    (4.0, 300.0),
    
    # LOG_INIT_LEARNINGRATE
    (1.0e-4, 1.0e-2),
    
    # DropoutValue
    (0.0, 0.1)
]

dimensions_5 =[
    # NodesInFirstDense
    (4.0, 300.0),
    
    # NodesInSecondDense
    (4.0, 300.0),
    
    # NodesInThirdDense
    (4.0, 300.0),
    
    # LOG_INIT_LEARNINGRATE
    (1.0e-4, 1.0e-2),
    
    # DropoutValue
    (0.0, 0.1)
]

dimensions_6 =[
    # NodesInFirstDense
    (4.0, 300.0),
    
    # NodesInSecondDense
    (4.0, 300.0),
    
    # NodesInThirdDense
    (4.0, 300.0),
    
    # NodesInFourthDense
    (4.0, 300.0),
    
    # LOG_INIT_LEARNINGRATE
    (1.0e-4, 1.0e-2),
    
    # DropoutValue
    (0.0, 0.1)
]

dimensions_7 =[
    # NodesInFirstDense
    (4.0, 300.0),
    
    # NodesInSecondDense
    (4.0, 300.0),
    
    # NodesInThirdDense
    (4.0, 300.0),
    
    # NodesInFourthDense
    (4.0, 300.0),
    
    # NodesInFifthDense
    (4.0, 300.0),
    
    # LOG_INIT_LEARNINGRATE
    (1.0e-4, 1.0e-2),
    
    # DropoutValue
    (0.0, 0.1)
]

In [112]:
### Random forest regressor optimizer ###

bo_rf_41 = Optimizer(
    dimensions=dimensions_4,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=1,
    acq_func='EI',   
)

In [113]:
bo_rf_42 = Optimizer(
    dimensions=dimensions_5,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=1,
    acq_func='EI',   
)

In [114]:
bo_rf_43 = Optimizer(
    dimensions=dimensions_6,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=1,
    acq_func='EI',
)

In [115]:
bo_rf_44 = Optimizer(
    dimensions=dimensions_7,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=1,
    acq_func='EI',   
)

In [6]:
ResultsDict = {}
for i in range(0,3):
    OptimalParams = ([])
    OptimalScore = ([])
    
    bo_rf_41 = Optimizer(
    dimensions=dimensions_4,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=10,
    acq_func='EI',   
    )

    bo = bo_rf_41

    for j in range(100):
        x = bo.ask()
        print(x)
        f = target_function1(x) # Other inputs are automatically set

        bo.tell(x, f)

        plot_convergence(bo)

    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]

    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    FirstNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}

ResultsDict['First network'] = FirstNet
# Second network, three dense layers
for i in range(0,3):
    
    OptimalParams = ([])
    OptimalScore = ([])

    bo_rf_42 = Optimizer(
    dimensions=dimensions_5,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=10,
    acq_func='EI',   
    )

    bo = bo_rf_42

    for j in range(150):
        x = bo.ask()
        print(x)
        f = target_function2(x) # Other inputs are automatically set

        bo.tell(x, f)

        plot_convergence(bo)

    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]


    OptimalParams = np.append(OptimalParams, best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    SecondNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}

ResultsDict['Second network'] = SecondNet
#Third network, four dense layers
for i in range(0,3):
    
    OptimalParams = ([])
    OptimalScore = ([])

    bo_rf_43 = Optimizer(
    dimensions=dimensions_6,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=10,
    acq_func='EI',   
    )

    bo = bo_rf_43

    for j in range(180):
        x = bo.ask()
        print(x)
        f = target_function3(x) # Other inputs are automatically set

        bo.tell(x, f)

        plot_convergence(bo)

    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]

    OptimalParams = np.append(OptimalParams, best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    ThirdNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}

ResultsDict['Third network'] = ThirdNet
# fourth network, five dense layers
for i in range(0,3):
    
    OptimalParams = ([])
    OptimalScore = ([])

    bo_rf_44 = Optimizer(
    dimensions=dimensions_7,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=10,
    acq_func='EI',   
    )

    bo = bo_rf_44

    for j in range(200):
        x = bo.ask()
        print(x)
        f = target_function4(x) # Other inputs are automatically set

        bo.tell(x, f)

        plot_convergence(bo)

    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]

    OptimalParams = np.append(OptimalParams, best_parameters, axis = 0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    FourthNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}

ResultsDict['Fourth network'] = FourthNet
# fifth network, No batch normalization
for i in range(0,3):    
    
    OptimalParams = ([])
    OptimalScore = ([])

    bo_rf_41 = Optimizer(
    dimensions=dimensions_4,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6),
    n_initial_points=10,
    acq_func='EI',   
    )

    bo = bo_rf_41

    for j in range(100):
        x = bo.ask()
        print(x)
        f = target_function_NoBatch(x) # Other inputs are automatically set

        bo.tell(x, f)

        plot_convergence(bo)

    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]

    OptimalParams = np.append(OptimalParams, best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams, best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    bo.yi[np.argmin(bo.yi)] = 1
    
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]
    
    OptimalParams = np.append(OptimalParams,best_parameters, axis=0)
    OptimalScore = np.append(OptimalScore,bo.yi[np.argmin(bo.yi)])
    
    FifthNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}

ResultsDict['Fifth network'] = FifthNet

NameError: name 'Optimizer' is not defined

In [50]:
OptimalScore3

array([0.63773383, 0.64893887, 1.        ])

In [34]:
best_result_index = np.argmin(bo.yi)
best_parameters = bo.Xi[best_result_index]

In [35]:
bo.yi[np.argmin(bo.yi)]

0.5384148328829712

In [36]:
bo.yi[np.argmin(bo.yi)] = 1

In [37]:
bo.yi[np.argmin(bo.yi)]

0.5414268371482693

In [39]:
best_result_index = np.argmin(bo.yi)
best_parameters = bo.Xi[best_result_index]
best_parameters

[173.2773707552993,
 14.45812940532587,
 0.007589349274725483,
 0.008446173333937724]

In [30]:
np.append(OptimalScore1,bo.yi[np.argmin(bo.yi)])

array([0.53841483])

In [23]:
best_result_index = np.argmin(bo.yi)
best_parameters = bo.Xi[best_result_index]

best_result_index
best_parameters
bo.yi[np.argmin(bo.yi)]

0.5900488114779627

In [ ]:
### Gaussian optimizer ###

bo_gp_1 = Optimizer(
    ### telling optimizer boundaries for each parameter
    dimensions=dimensions_1,
    
    ### setting regressor
    base_estimator=GaussianProcessRegressor(
        kernel=RBF(length_scale_bounds=[1.0e-6, 1.0e+6]) + \
            WhiteKernel(noise_level=1.0e-5, noise_level_bounds=[1.0e-6, 1.0e-2]),
    ),
    n_initial_points=2,
    acq_func='EI',   
)

In [37]:
### Actual optimization process ###
def Optimization(bo,target_function):
    Parameter_List = ([])
    for i in range(1):
        x = bo.ask()
        print(x)
        f = target_function(x) # Other inputs are automatically set

        Parameter_List = np.append(Parameter_List, x)
        bo.tell(x, f)

        plot_convergence(bo)
        
        best_result_index = np.argmin(bo.yi)
        best_parameters = bo.Xi[best_result_index]

In [21]:
print_best(bo_rf_4)

Best model:
 Nodes in first dense layer= 235 
 Nodes in second dense layer= 147 
 learning rate= 0.02177968898175681 
 Dropout value= 0.005580370632707155


In [9]:
### Testing the found parameters to see if the results are reproduceable ###

INIT_LEARNINGRATE = 0.02532767329545395  
BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
EPOCHS = 3

assert len(y_train) == len(x_train), "x_train and y_train not same length!"
#assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

K.clear_session()

model = Sequential()
model.add(Dense(271, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
model.add(BatchNormalization())
model.add(Dropout(0.0009682674694188368))
model.add(Dense(280, activation='relu'))
model.add(Dense( len(classes) )) # muon and 'other'
model.add(Activation("softmax")) # output probabilities

print(model.summary())

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
    metrics=['accuracy'] 
    )

# Running test for 5 times in this case
for i in range(0,5):
    model.fit(
        x_train, y_train,
        batch_size = BATCH_SIZE,
        epochs = EPOCHS,
        validation_data = (x_val, y_val),
        shuffle = True
        )


    #model.save_model("keras_basic_model.xgb")


    # score

    validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
    result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
    print(result)
    
    i += 1

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 271)               17886     
_________________________________________________________________
batch_normalization_1 (Batch (None, 271)               1084      
_________________________________________________________________
dropout_1 (Dropout)          (None, 271)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 280)               76160     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 562       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 95,692
Trainable params: 95,150
Non-trainable params: 542
_________________________________________________________________
None

In [17]:
#### Original keras model for reference ####

INIT_LEARNINGRATE = 5e-3
BATCH_SIZE = 16  # should be a factor of len(x_train) and len(x_val) etc.
EPOCHS = 3

assert len(y_train) == len(x_train), "x_train and y_train not same length!"
#assert len(y_train) % BATCH_SIZE == 0, "batch size should be multiple of training size,{0}/{1}".format(len(y_train),BATCH_SIZE)

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

K.clear_session()

model = Sequential()
model.add(Dense(100, activation='relu', input_shape=( len( utils.SIMPLE_FEATURE_COLUMNS ), ))) #length = input vars
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(50 , activation='relu'))
model.add(Dense( len(classes) )) # muon and 'other'
model.add(Activation("softmax")) # output probabilities

print(model.summary())

model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.adamax(lr=INIT_LEARNINGRATE),
    metrics=['accuracy'] 
    )


model.fit(
    x_train, y_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    validation_data = (x_val, y_val),
    shuffle = True
    )

#model.save_model("keras_basic_model.xgb")


# score

validation_predictions = model.predict_proba(val_part.loc[:, utils.SIMPLE_FEATURE_COLUMNS].values)[:, 1]
result = scoring.rejection90(val_part.label.values, validation_predictions, sample_weight=val_part.weight.values)
print(result)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               6600      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 12,152
Trainable params: 11,952
Non-trainable params: 200
_________________________________________________________________
None

In [ ]:
for i in range(0,5):
    
    for j in range(0,5):
        
        

In [7]:
import multiprocessing

In [8]:
!nproc


'nproc' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
pool = multiprocessing.Pool(processes=4)

In [18]:
r = pool.map(model.fit, [x_train, y_train])

ValueError: Error when checking input: expected dense_1_input to have shape (65,) but got array with shape (2,)

In [4]:
i = 1
DataDict = {}
DataDict = {'FirstNetwork' : {'run {0}'.format(i) : 1}}

In [5]:
DataDict['FirstNetwork']['run 1']

1

In [7]:
OptimalParams = np.array([9,3,2,3])
OptimalScore = np.array([8,3,3,3,3,2,2,2])
ResultsDict = {}

NameError: name 'np' is not defined

In [34]:
i = 'a'
# SecondNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}
ResultsDict = {'Second network' : { i : {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}}

i = 'b'

OptimalParams = np.array([1,3,2,3])
OptimalScore = np.array([1,3,3,3])


ResultsDict['Second network'][i] = {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}
#wordFreqDic.update(test = 'value' )
#ResultsDict['Second network'] = SecondNet

In [35]:
ResultsDict

{'Second network': {'a': {'Optimal Parameters': array([1, 3, 2, 3]),
   'Optimal Score': array([1, 3, 3, 3])},
  'b': {'Optimal Parameters': array([1, 3, 2, 3]),
   'Optimal Score': array([1, 3, 3, 3])}}}

In [14]:
i = 1
SecondNet = {'Run {0}'.format(i): {'Optimal Parameters' : OptimalParams, 'Optimal Score' : OptimalScore}}

ResultsDict['Second network'] = SecondNet

In [15]:
ResultsDict = {'Second network' : 

{'Run 1': {'Optimal Parameters': array([1, 3, 2, 3]),
  'Optimal Score': array([1, 3, 3, 3])}}

In [29]:
for i in ['a', 'b', 'c']:
    print(i)

a
b
c
